# **(Modeling And Evaluation)**

## Objectives

* Build and train a custom convolutional neural network (CNN) from scratch for tumor detection in CT scans.
* Tune hyperparameters to optimize model performance.
* Evaluate the model using accuracy, recall, and inference time metrics.
* Generate model predictions and confidence scores for downstream visualization.
* Prepare the model and outputs for integration with the Streamlit dashboard.

## Inputs

* Preprocessed and augmented image data and metadata from the DataCollection notebook.
* Train/validation/test splits.
* Any configuration files or parameters for model training.

## Outputs

* Trained custom CNN model (saved in a suitable format, e.g., .h5 or .pb).
* Evaluation metrics (accuracy, recall, inference time) and confusion matrix.
* Model predictions and confidence scores for each sample.
* Artifacts for dashboard integration (e.g., prediction results, model files).

## Additional Comments

* The model should be compact enough for real-time inference (<1.5 sec/sample).
* Early stopping and validation loss monitoring should be used to prevent overfitting.
* All outputs will be used in the DataVisualization notebook and Streamlit dashboard.

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/brain-tumor-classification/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir('/workspaces/brain-tumor-classification')
print("Current working directory:", os.getcwd())

Current working directory: /workspaces/brain-tumor-classification


Confirm the new current directory

**Environment Setup, Data loading and preparation**

Core libraries

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import random
import warnings

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

2025-06-22 19:08:55.895578: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-22 19:08:55.898190: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-22 19:08:55.903475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-22 19:08:55.920412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750619335.947019    4260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750619335.95

TensorFlow version: 2.19.0
Keras version: 3.10.0
Numpy version: 1.26.1
Pandas version: 2.1.1


Data Loading & Splitting

In [4]:
train_dir = "inputs/brain_tumor_dataset/images/train"
val_dir = "inputs/brain_tumor_dataset/images/val"
test_dir = "inputs/brain_tumor_dataset/images/test"

In [5]:
def count_images(directory):
    total = 0
    for label in os.listdir(directory):
        class_path = os.path.join(directory, label)
        if os.path.isdir(class_path):
            total += len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return total

print("Train images:", count_images(train_dir))
print("Validation images:", count_images(val_dir))
print("Test images:", count_images(test_dir))

Train images: 7030
Validation images: 1054
Test images: 1054


---

Data Preparation & Normalization

Define Utility Function for tf.data Pipeline

In [6]:

IMG_SIZE = (224, 224)
BATCH_SIZE = 8



Build File Path and Label Lists

In [7]:
import glob

def get_file_paths_and_labels(data_dir):
    class_names = sorted(os.listdir(data_dir))
    file_paths = []
    labels = []
    for idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            files = glob.glob(os.path.join(class_dir, '*'))
            file_paths.extend(files)
            labels.extend([idx] * len(files))
    return file_paths, labels, class_names

train_files, train_labels, class_names = get_file_paths_and_labels(train_dir)
val_files, val_labels, _ = get_file_paths_and_labels(val_dir)
test_files, test_labels, _ = get_file_paths_and_labels(test_dir)

neg, pos = np.bincount(train_labels)
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)  
weight_for_1 = (1 / pos) * (total / 2.0) 

class_weights = {0: weight_for_0, 1: weight_for_1}
print("Class weights:", class_weights)

print("Classes:", class_names)
print("Train samples:", len(train_files))
print("Validation samples:", len(val_files))
print("Test samples:", len(test_files))

Class weights: {0: 1.0, 1: 1.0}
Classes: ['notumor', 'tumor']
Train samples: 7030
Validation samples: 1054
Test samples: 1054


because all images are PNG , this is a result of a debug. (34)

In [8]:
def preprocess_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

Create tf.data Datasets

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))

train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-06-22 19:09:05.458577: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Confirm Class Balance in Training Set

In [10]:
import numpy as np

unique, counts = np.unique(train_labels, return_counts=True)
class_balance = dict(zip(class_names, counts))
print("Class balance in training set:", class_balance)

Class balance in training set: {'notumor': 3515, 'tumor': 3515}


---

Model Architecture Design

In [11]:
from tensorflow.keras import layers, models

def build_custom_cnn(input_shape=(224, 224, 3)):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Normalization(),  # Input normalization layer

        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

model = build_custom_cnn()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 224, 224, 3)    │             7 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,464 (49.36 MB)

 Trainable params: 12,939,009 (49.36 MB)

 Non-trainable params: 455 (1.78 KB)

Model Compilation.

Choose an appropriate optimizer (e.g., Adam), loss function (e.g., binary crossentropy), and evaluation metrics (accuracy, recall).
Compile the model with these settings.

Compile the Model

In [12]:
from tensorflow.keras import optimizers, metrics

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', metrics.Recall()]
)

Run this to check if tensorflow could not be resolved error means tensorflow is not installed in your instance, in this case it is installed and we have no problem .

In [13]:
import sys
print(sys.executable)

/home/cistudent/.pyenv/versions/3.12.1/bin/python


# Recall-Focused Model Training.

Train the model on the training set, validating on the validation set.
Use callbacks such as EarlyStopping and ModelCheckpoint, monitoring validation recall.
Log training and validation metrics for each epoch.
Track the precision-recall tradeoff.


Set Up Callbacks

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_recall', patience=5, mode='max', restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.keras', monitor='val_recall', mode='max', save_best_only=True, verbose=1)
]

In [15]:
with open(".gitignore", "a") as f:
    f.write("\n# Ignore model files\n*.h5\n*.keras\nbest_model.h5\nmy_model.keras\n")

In [16]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/30
879/879 ━━━━━━━━━━━━━━━━━━━━ 0s 776ms/step - accuracy: 0.9716 - loss: 0.2275 - recall: 0.5440
Epoch 1: val_recall improved from -inf to 0.99602, saving model to best_model.keras
879/879 ━━━━━━━━━━━━━━━━━━━━ 714s 804ms/step - accuracy: 0.9716 - loss: 0.2276 - recall: 0.5445 - val_accuracy: 0.8454 - val_loss: 0.3816 - val_recall: 0.9960
Epoch 2/30
879/879 ━━━━━━━━━━━━━━━━━━━━ 0s 835ms/step - accuracy: 0.9412 - loss: 0.1700 - recall: 0.5581
Epoch 2: val_recall improved from 0.99602 to 1.00000, saving model to best_model.keras
879/879 ━━━━━━━━━━━━━━━━━━━━ 761s 859ms/step - accuracy: 0.9412 - loss: 0.1699 - recall: 0.5586 - val_accuracy: 0.7989 - val_loss: 0.7888 - val_recall: 1.0000
Epoch 3/30
879/879 ━━━━━━━━━━━━━━━━━━━━ 0s 592ms/step - accuracy: 0.9522 - loss: 0.1489 - recall: 0.5694
Epoch 3: val_recall did not improve from 1.00000
879/879 ━━━━━━━━━━━━━━━━━━━━ 576s 607ms/step - accuracy: 0.9523 - loss: 0.1489 - recall: 0.5699 - val_accuracy: 0.8558 - val_loss: 0.5516 - val_rec

In [17]:
model.save("my_model.keras")
print("Model saved locally as my_model.keras")

Model saved locally as my_model.keras
